In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc 

from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import matplotlib.image as img
import seaborn as sns
%matplotlib inline
import numpy as np 
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
import sklearn
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import ADASYN 

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# имортируем метрики для оценки классифкаторов
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
from sklearn.svm import LinearSVC
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.preprocessing import RobustScaler

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error as mse, r2_score as r2

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import warnings

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow
# NN folds
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import StratifiedKFold

warnings.filterwarnings('ignore')

#https://www.kaggle.com/camnugent/geospatial-feature-engineering-and-visualization

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif

                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
df = reduce_mem_usage(pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/train.csv', index_col = 'Id'))
# df2 = reduce_mem_usage(pd.read_csv('test.csv', index_col = 'Id')).join(pd.read_csv('target1.csv', index_col = 'Id'))

# df = pd.concat([df,df2])
df_final = reduce_mem_usage(pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/test.csv', index_col = 'Id'))

In [ ]:
df = df[df['Cover_Type'] != 5]

df_copy = df.sample(frac=1)

X = df_copy.drop('Cover_Type', axis=1)
y = df_copy['Cover_Type']

X_final = df_final.copy()

In [ ]:
%%time
print('start')
# NNFolds
N_SPLITS = 5

### cross-validation 
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=50)

scores = {fold:None for fold in range(cv.n_splits)}
predictions = []

for fold, (idx_train, idx_valid) in enumerate(cv.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    XX_final = X_final.copy()

    
    cols_scale = X.iloc[:,:-44].columns

    scaler = StandardScaler()

    X_train[cols_scale] = scaler.fit_transform(X_train[cols_scale])
    X_valid[cols_scale] = scaler.transform(X_valid[cols_scale])
    XX_final[cols_scale] = scaler.transform(XX_final[cols_scale])
    
    with tensorflow.device('/gpu:0'):
        
        print('start2')
        model = keras.Sequential([ 
            layers.BatchNormalization(input_shape=[54]),
            layers.Dense(128, activation='swish'),
            layers.Dropout(0.03),
            layers.BatchNormalization(),
            layers.Dense(128, activation='swish'),
            layers.Dropout(0.03),
            layers.BatchNormalization(),
            layers.Dense(128, activation='swish'),
            layers.Dropout(0.03),
            layers.BatchNormalization(),
            layers.Dense(128, activation='swish'),
            layers.Dropout(0.03),
            layers.BatchNormalization(),
            layers.Dense(128, activation='swish'),
            layers.Dropout(0.03),
            layers.BatchNormalization(),
            layers.Dense(7, activation='softmax')
        ])

        model.compile(
                    optimizer=keras.optimizers.Adam(learning_rate=0.0003,beta_1=0.99,beta_2=0.999),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'],
                )
        lr = ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, 
                                   patience=10, verbose=0)
        early_stopping = keras.callbacks.EarlyStopping(
            monitor = 'val_accuracy',
            mode = 'max',
            patience=20,
            restore_best_weights=True,
        )

        history = model.fit(
            X_train, y_train-1,
            validation_data=(X_valid, y_valid-1),
            batch_size=5000,
            epochs=1000,
            callbacks=[early_stopping,lr],
            verbose=0 # hide the output because we have so many epochs
        )

    preds1 = model.predict(X_valid)
    df_NN1 = pd.DataFrame(preds1, columns = [1,2,3,4,5,6,7])
    preds_y = df_NN1.idxmax(axis=1)
    preds_y.index = y_valid.index
    print('accuracy:',accuracy_score(y_valid,preds_y))

    print('**'*20)
    print(f"Fold {fold+1} || Training")
    print('**'*20)

    scores[fold] = (history.history)

    prediction = model.predict(XX_final)
    predictions.append(prediction)

print('**'*20)
print('Finished Training')
print('**'*20)

In [ ]:
pred_sum = predictions[0]

for i in range(1,5):
    pred_sum = pred_sum + predictions[i]
    
pred_each = pred_sum/5
df_NN = pd.DataFrame(pred_each, columns = ['1','2','3','4','5','6','7'])
df_NN.index = X_final.index
df_NN.to_csv('NN_prob.csv')

df_out = pd.DataFrame(df_NN.idxmax(axis=1), columns = ['Cover_Type'])
df_out['Id'] = X_final.index
columns = ['Id','Cover_Type']
df_out.to_csv('target.csv',index=None)

df_out[columns]